In [ ]:
%load_ext dotenv
%dotenv
%load_ext lab_black
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from common import (
    format_ax,
    query_mssql_iteratively,
    create_emission_source_query,
    get_sample_of_peak_measurements_from_each_analyzer,
)
from config import CUSTOMER

In [ ]:
CUSTOMER

## Get Emission Sources

In [ ]:
emission_sources = pd.read_pickle(
    f"data/non-reprocessed-emission-sources-{CUSTOMER}.pickle"
)
emission_sources.info()
emission_sources.head()

## Check if Lateral Wind Correction is Needed

In [ ]:
%%time

analyzer_measurement_samples = get_sample_of_peak_measurements_from_each_analyzer(emission_sources, sample_fraction=0.4)

In [ ]:
# analyzer_measurement_samples


In [ ]:
fig, axs = plt.subplots(
    nrows=len(analyzer_measurement_samples),
    figsize=(12, 10 * len(analyzer_measurement_samples)),
)

for i, (analyzer_id, measurements_sample) in enumerate(
    analyzer_measurement_samples.items()
):
    if type(axs) != np.ndarray:
        ax = axs
    else:
        ax = axs[i]

    try:
        measurements_sample.WeatherStationRotation.hist(ax=ax, bins=50)
        format_ax(
            ax=ax,
            title=f"Analyzer: {analyzer_id}\nmedian={round(measurements_sample.WeatherStationRotation.median(),2)}",
            x_label="WeatherStationRotation (degrees)",
            y_label="Measurement Count",
        )
    except AttributeError:
        pass


fig.tight_layout()

In [ ]:
fig, axs = plt.subplots(
    nrows=len(analyzer_measurement_samples),
    figsize=(12, 10 * len(analyzer_measurement_samples)),
)

for i, (analyzer_id, measurements_sample) in enumerate(
    analyzer_measurement_samples.items()
):
    if type(axs) != np.ndarray:
        ax = axs
    else:
        ax = axs[i]

    measurements_sample = measurements_sample.sort_values(by="EpochTime")

    ax.plot(
        measurements_sample["EpochTime"],
        measurements_sample["WeatherStationRotation"],
        label="weather station rotation",
    )

    ax = format_ax(
        ax=ax,
        title=f"{CUSTOMER.title()} measurements_sample",
        x_label="epoch seconds",
        y_label="degrees",
    )

    ax.set_ylim((-20, 20))


fig.tight_layout()